# Data Science Capstone

## Comparing Memphis and Portland's access to fitness facilities, and food venues


In [3]:
!conda install -c conda-forge geopy --yes        # if needed
!conda install -c conda-forge folium=0.5.0 --yes # if needed

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values



import urllib.request
from bs4 import BeautifulSoup
import requests


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Scraping Wikipedia for Memphis and Portland Neighbourhoods

## Memphis Neighbourhoods

In [132]:
url = 'https://raw.githubusercontent.com/Daylen-Mackey/Data_Science_Capstone/master/MemphisNeighbourhoods.csv'
mNeigh = pd.read_csv(url)
mNeigh.head(20)

,Neighbourhoods
0,Central Business District
1,Edge District
2,Harbor Town
3,Linden
4,Medical District
5,Pinch District
6,South Forum
7,South Main Arts District
8,Speedway Terrace
9,Uptown/Greenlaw


### Some Memphis Neighbourhoods have a '/' in them. Let's split that -- duplicates can be filtered out later

### *Before*


In [133]:
#before
mNeigh.tail()

,Neighbourhoods
102,Fairgrounds
103,Glenview
104,Lamar Avenue
105,Poplar Avenue
106,Union Extended


In [135]:
replicates = mNeigh[mNeigh.Neighbourhoods.str.contains('/')]
replicates

,Neighbourhoods
9,Uptown/Greenlaw
32,Balmoral/Quince
34,Cherry/Willow
37,Galloway Gardens/Walnut Grove


In [136]:
replicates['Neighbourhoods'] = replicates['Neighbourhoods'].apply(lambda x: x.split('/')[1]) 
replicates

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Neighbourhoods
9,Greenlaw
32,Quince
34,Willow
37,Walnut Grove


### Remove the repllicates and append to bottom of data frame

In [137]:
#Keep what's on the left side of the '/'
mNeigh['Neighbourhoods'] = mNeigh['Neighbourhoods'].apply(lambda x: x.split('/')[0]) 


In [138]:
copy = mNeigh
mCopy = pd.concat([copy, replicates])


### *After*


In [143]:

mCopy.reset_index(drop = True, inplace = True)
mCopy.tail(15)

,Neighbourhoods
96,Magnolia
97,Oakhaven
98,Orange Mound
99,Parkway Village
100,Riverdale
101,Southwind
102,Fairgrounds
103,Glenview
104,Lamar Avenue
105,Poplar Avenue


## Portland Neighbourhoods

In [3]:
url = 'https://raw.githubusercontent.com/Daylen-Mackey/Data_Science_Capstone/master/PortlandNeighbourhoods.csv'
pNeigh = pd.read_csv(url)
pNeigh.head(15)

,Neighbourhoods
0,Arlington Heights
1,Forest Park
2,Goose Hollow
3,Hillside
4,Linnton
5,"Northwest District (includes Uptown, Nob Hill,..."
6,Northwest Heights
7,Northwest Industrial
8,Old Town Chinatown
9,Pearl District


### Portland Dataframe contains (includes...) many times -- need to remove

In [4]:
Pcopy = pNeigh
Pcopy['Neighbourhoods'] = Pcopy['Neighbourhoods'].apply(lambda x: x.split('(')[0])
Pcopy.head(20)

,Neighbourhoods
0,Arlington Heights
1,Forest Park
2,Goose Hollow
3,Hillside
4,Linnton
5,Northwest District
6,Northwest Heights
7,Northwest Industrial
8,Old Town Chinatown
9,Pearl District


In [32]:
Pcopy['Name'] = Pcopy['Neighbourhoods'] + ', Portland, Oregon'
Pcopy.size

190

In [33]:
geolocator = Nominatim(user_agent="ny_explorer")

pLat = []
pLong = []

for city in Pcopy.Name:
    location = geolocator.geocode(city)
    try:
            pLat.append(location.latitude)
            pLong.append(location.longitude)
    except:
        pLat.append('NaN')
        pLong.append('NaN')
    #pLong = pLong.append(location.latitude)
    

In [40]:
Pcopy['Latitude'] = pLat
Pcopy['Longitude'] = pLong
Pcopy

,Neighbourhoods,Name,Latitude,Longitude
0,Arlington Heights,"Arlington Heights, Portland, Oregon",45.5195,-122.711
1,Forest Park,"Forest Park, Portland, Oregon",45.5615,-122.759
2,Goose Hollow,"Goose Hollow, Portland, Oregon",45.5177,-122.693
3,Hillside,"Hillside, Portland, Oregon",45.5274,-122.713
4,Linnton,"Linnton, Portland, Oregon",45.6003,-122.787
5,Northwest District,"Northwest District , Portland, Oregon",45.533,-122.699
6,Northwest Heights,"Northwest Heights, Portland, Oregon",45.5408,-122.774
7,Northwest Industrial,"Northwest Industrial, Portland, Oregon",NaN,NaN
8,Old Town Chinatown,"Old Town Chinatown, Portland, Oregon",45.5249,-122.674
9,Pearl District,"Pearl District, Portland, Oregon",45.529,-122.682


In [43]:
mNeigh['Name'] = mNeigh['Neighbourhoods'] + ', Memphis, Tennessee'
mNeigh

,Neighbourhoods,Name
0,Central Business District,"Central Business District, Memphis, Tennessee"
1,Edge District,"Edge District, Memphis, Tennessee"
2,Harbor Town,"Harbor Town, Memphis, Tennessee"
3,Linden,"Linden, Memphis, Tennessee"
4,Medical District,"Medical District, Memphis, Tennessee"
5,Pinch District,"Pinch District, Memphis, Tennessee"
6,South Forum,"South Forum, Memphis, Tennessee"
7,South Main Arts District,"South Main Arts District, Memphis, Tennessee"
8,Speedway Terrace,"Speedway Terrace, Memphis, Tennessee"
9,Uptown/Greenlaw,"Uptown/Greenlaw, Memphis, Tennessee"


In [47]:
geolocator = Nominatim(user_agent="ny_explorer")

mLat = []
mLong = []

for city in mNeigh['Name']:
    location = geolocator.geocode(city)
    try:
        mLat.append(location.latitude)
        mLong.append(location.longitude)
    except:
        mLat.append('NaN')
        mLong.append('NaN')
    #pLong = pLong.append(location.latitude)

In [49]:
mNeigh['Latitude'] = mLat
mNeigh['Longitude'] = mLong
mNeigh

,Neighbourhoods,Name,Latitude,Longitude
0,Central Business District,"Central Business District, Memphis, Tennessee",NaN,NaN
1,Edge District,"Edge District, Memphis, Tennessee",35.1598,-90.0543
2,Harbor Town,"Harbor Town, Memphis, Tennessee",35.1639,-90.0535
3,Linden,"Linden, Memphis, Tennessee",35.1348,-90.0273
4,Medical District,"Medical District, Memphis, Tennessee",35.142,-90.0303
5,Pinch District,"Pinch District, Memphis, Tennessee",NaN,NaN
6,South Forum,"South Forum, Memphis, Tennessee",NaN,NaN
7,South Main Arts District,"South Main Arts District, Memphis, Tennessee",NaN,NaN
8,Speedway Terrace,"Speedway Terrace, Memphis, Tennessee",35.1556,-90.0198
9,Uptown/Greenlaw,"Uptown/Greenlaw, Memphis, Tennessee",NaN,NaN


### Hidden Cell -- Connecting to FourSquare API

In [5]:
# The code was removed by Watson Studio for sharing.

### Finding the Coordinates of Portland, Oregon and Memphis, Tennessee

In [6]:
city = 'Portland, Oregon'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city)
portland_latitude = location.latitude
portland_longitude = location.longitude

city = 'Memphis, Tennessee'

location = geolocator.geocode(city)
memphis_latitude = location.latitude
memphis_longitude = location.longitude


print('The geograpical coordinate of Portland are {}, {}.'.format(portland_latitude, portland_longitude))
print('The geograpical coordinate of Memphis are {}, {}.'.format(memphis_latitude, memphis_longitude))

The geograpical coordinate of Portland are 45.5202471, -122.6741949.
The geograpical coordinate of Memphis are 35.1490215, -90.0516285.


## Relevant Forsquare Categories
Gym / Fitness Center
4bf58dd8d48988d175941735

Grocery Store
4bf58dd8d48988d118951735

Health Food Store
50aa9e744b90af0d42d5de0e

Fast Food Restaurant
4bf58dd8d48988d16e941735


In [7]:
radius = 50
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, portland_latitude, portland_longitude, VERSION, radius, LIMIT)


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
results = requests.get(url).json()["response"]['groups'][0]['items']
venues_list = []
venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

NameError: name 'name' is not defined

In [ ]:
results
